Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        print(data.get('utc'))
        self.log_all(orders, data, ctx, data.get('utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [3]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
asset = Asset(c.ETH, c.BTC)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_backtest'

In [4]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-10T14:00:00
---------------------------------------
OHLCV
    O: 0.0964 | C: 0.0966 | V: 8929.1 | T: 2018-01-10T14:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0966 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-10T14:30:00
---------------------------------------
OHLCV
    O: 0.0966 | C: 0.0965 | V: 9865.7 | T: 2018-01-10T14:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9952 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99517104999999995, 'used': 0.0, 'total': 0.99517104999999995}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.096578999999

2018-01-10 14:00:00
Equal? True
2018-01-10 14:30:00
Equal? True
2018-01-10 15:00:00
Order Failed {
    "id": "e6532c2213f44ccfb1beb4fc8cab9a9f",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-21T06:07:02.898564",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 0,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00479. BTC balance is: 0.99999405. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
2018-01-10 15:30:00
Order Failed {
    "id": "e6532c2213f44ccfb1beb4fc8cab9a9f",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "cr

    BTC - {'free': 0.98629884999999995, 'used': 0.0, 'total': 0.98629884999999995}
    ETH - {'free': 0.15000000000000002, 'used': 0.0, 'total': 0.15000000000000002}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.15000000000000002, 'cost_price': 0.089823166666666662, 'latest_price': 0.088610000000000008}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 13 - Timestep: 2018-01-10T20:00:00
---------------------------------------
OHLCV
    O: 0.0908 | C: 0.0901 | V: 6086.7 | T: 2018-01-10T20:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9818 Total Val: 0.9999 PnL: -0.0001 Returns: -0.0001
BALANCE
    BTC - {'free': 0.98175159999999995, 'used': 0.0, 'total': 0.98175159999999995}
    ETH - {'free': 0.2, 'used': 0.0, 'total': 0.2}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.2, 'cost_price': 0.090103625000000007, 'latest_price': 0.090944999999999998}
METRICS
    SMA: 5.00
    RSI: 10.00
----------

Equal? True
2018-01-10 20:00:00
Equal? True
2018-01-10 20:30:00
Equal? True
2018-01-10 21:00:00
Equal? True
2018-01-10 21:30:00
Equal? True
2018-01-10 22:00:00
Equal? True
2018-01-10 22:30:00
Equal? True
2018-01-10 23:00:00
Equal? True
2018-01-10 23:30:00
Equal? True
2018-01-11 00:00:00
Equal? True
2018-01-11 00:30:00
Equal? True


---------------------------------------
Epoch 23 - Timestep: 2018-01-11T01:00:00
---------------------------------------
OHLCV
    O: 0.0915 | C: 0.0896 | V: 8795.0 | T: 2018-01-11T01:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0896 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9819 Total Val: 1.0002 PnL: 0.0002 Returns: 0.0002
BALANCE
    BTC - {'free': 0.98191394999999992, 'used': 0.0, 'total': 0.98191394999999992}
    ETH - {'free': 0.2, 'used': 0.0, 'total': 0.2}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.2, 'cost_price': 0.088009467773437516, 'latest_price': 0.091497000000000009}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 24 - Timestep: 2018-01-11T01:30:00
---------------------------------------
OHLCV
    O: 0.0895 | C: 0.0898 | V: 5096.1 | T: 2018-01-11T01:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9774 Total Val: 0.9998 PnL: -0.0002 Returns: -0.0002
BALANCE
    BTC - {'fr

...

2018-01-11 13:30:00
Equal? True
2018-01-11 14:00:00
Equal? True
2018-01-11 14:30:00
Equal? True
2018-01-11 15:00:00
Equal? True
2018-01-11 15:30:00
Equal? True
2018-01-11 16:00:00
Equal? True
2018-01-11 16:30:00
Equal? True
2018-01-11 17:00:00
Equal? True
2018-01-11 17:30:00


    O: 0.0888 | C: 0.0893 | V: 6388.7 | T: 2018-01-11T17:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9686 Total Val: 0.9997 PnL: -0.0003 Returns: -0.0003
BALANCE
    BTC - {'free': 0.96861309999999978, 'used': 0.0, 'total': 0.96861309999999978}
    ETH - {'free': 0.35, 'used': 0.0, 'total': 0.35}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.35, 'cost_price': 0.088114783163265334, 'latest_price': 0.088882000000000003}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 57 - Timestep: 2018-01-11T18:00:00
---------------------------------------
OHLCV
    O: 0.0893 | C: 0.0893 | V: 7204.2 | T: 2018-01-11T18:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0893 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9731 Total Val: 0.9999 PnL: -0.0001 Returns: -0.0001
BALANCE
    BTC - {'free': 0.97308039999999973, 'used': 0.0, 'total': 0.97308039999999973}
    ETH - {'free': 0.3, 'used': 0.0, 'total': 0.3}
PO

Equal? True
2018-01-11 18:00:00
Equal? True
2018-01-11 18:30:00
Equal? True
2018-01-11 19:00:00
Equal? True
2018-01-11 19:30:00
Equal? True
2018-01-11 20:00:00
Equal? True
2018-01-11 20:30:00
Equal? True
2018-01-11 21:00:00
Equal? True
2018-01-11 21:30:00


PERFORMANCE
    Cash: 0.9775 Total Val: 0.9994 PnL: -0.0006 Returns: -0.0006
BALANCE
    BTC - {'free': 0.97745694999999966, 'used': 0.0, 'total': 0.97745694999999966}
    ETH - {'free': 0.25, 'used': 0.0, 'total': 0.25}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.25, 'cost_price': 0.088120435335276998, 'latest_price': 0.087802999999999992}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 65 - Timestep: 2018-01-11T22:00:00
---------------------------------------
OHLCV
    O: 0.0876 | C: 0.0880 | V: 5223.5 | T: 2018-01-11T22:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0880 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9818 Total Val: 0.9994 PnL: -0.0006 Returns: -0.0006
BALANCE
    BTC - {'free': 0.98183664999999964, 'used': 0.0, 'total': 0.98183664999999964}
    ETH - {'free': 0.2, 'used': 0.0, 'total': 0.2}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.2, 'cost_price': 0.088120435335276998, 'latest_price': 0.087594000000000005}
METRICS
 

Equal? True
2018-01-11 22:00:00
Equal? True
2018-01-11 22:30:00
Equal? True
2018-01-11 23:00:00
Equal? True
2018-01-11 23:30:00
Equal? True
No data after prior poll: 2018-01-11 23:30:00


## Simulate Orders

In [5]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(c.ETH, c.BTC), Asset(c.LTC, c.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT


---------------------------------------
Epoch 1 - Timestep: 2018-01-20T21:49:00
---------------------------------------
OHLCV
    O: 0.0897 | C: 0.0898 | V: 115.0 | T: 2018-01-20T21:49:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0898 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 500
2018-01-20 21:49:00
Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT


---------------------------------------
Epoch 2 - Timestep: 2018-01-20T21:50:00
---------------------------------------
OHLCV
    O: 0.0898 | C: 0.0897 | V: 292.1 | T: 2018-01-20T21:50:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0897 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9955 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99550970000000005, 'used': 0.0, 'total': 0.99550970000000005}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.089805999999999997, 'latest_price': 0.089805999999999997}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 500
2018-01-20 21:50:00
Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 499
Downloading: BTC/USDT


...

2018-01-21 05:41:00
Equal? True
Downloading: ETH/BTC
Downloaded rows: 36
Downloading: LTC/BTC
Downloaded rows: 36
Downloading: BTC/USDT


---------------------------------------
Epoch 474 - Timestep: 2018-01-21T05:42:00
---------------------------------------
OHLCV
    O: 0.0902 | C: 0.0902 | V: 130.4 | T: 2018-01-21T05:42:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9633 Total Val: 0.9994 PnL: -0.0006 Returns: -0.0006
BALANCE
    BTC - {'free': 0.9632898999999997, 'used': 0.0, 'total': 0.9632898999999997}
    ETH - {'free': 0.39999999999999986, 'used': 0.0, 'total': 0.39999999999999986}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.39999999999999986, 'cost_price': 0.090238521506261396, 'latest_price': 0.090246000000000007}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 36
2018-01-21 05:42:00
Equal? True
Downloading: ETH/BTC
Downloaded rows: 35
Downloading: LTC/BTC
Downloaded rows: 35
Downloading: BTC/USDT


---------------------------------------
Epoch 475 - Timestep: 2018-01-21T05:43:00
---------------------------------------
OHLCV
    O: 0.0902 | C: 0.0902 | V: 90.6 | T: 2018-01-21T05:43:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9678 Total Val: 0.9994 PnL: -0.0006 Returns: -0.0006
BALANCE
    BTC - {'free': 0.96779019999999971, 'used': 0.0, 'total': 0.96779019999999971}
    ETH - {'free': 0.34999999999999987, 'used': 0.0, 'total': 0.34999999999999987}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.34999999999999987, 'cost_price': 0.090238521506261396, 'latest_price': 0.090220000000000009}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 35
2018-01-21 05:43:00


In [ ]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(c.BINANCE)
cash = exchange.fetch_balance().get(c.BTC)[BalanceType.FREE]
print(c.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)

In [ ]:
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)